In [1]:
from bs4 import BeautifulSoup
import pandas as pd
import requests

import cv2
from PIL import Image
import pytesseract
import re
import numpy as np


from pyzbar.pyzbar import decode

import requests
import json

In [2]:
# Scrapping de la web e-aditivos.com para sacar la info de los aditivos.

URL = "https://e-aditivos.com/"

response = requests.get(URL)
soup = BeautifulSoup(response.text, "html.parser")

table = soup.find("table")

output_rows = []
for table_row in table.findAll('tr'):
    columns = table_row.findAll('td')
    output_row = []
    for column in columns:
        output_row.append(column.text)
    output_rows.append(output_row)

In [5]:
# Creación del DataFrame.

aditivos= pd.DataFrame(output_rows,columns=['id','tipo','origen','aditivo','classif'])[1:]
aditivos.head(5)

aditivos['id'] = aditivos['id'].str.strip()

aditivos_obj = aditivos.select_dtypes(['object'])
aditivos[aditivos_obj.columns] = aditivos_obj.apply(lambda x: x.str.strip())

aditivos['id'].replace('E-','E', regex=True, inplace=True)


(271, 5)

In [6]:
# Foto con el móvil con la app "IP WEBCAM" a través de la IP.

url = 'http://192.168.43.203:8080/video'
cap = cv2.VideoCapture(url)
while(True):
    ret, frame = cap.read()
    if frame is not None:
        cv2.imshow('frame',frame)
        cv2.imwrite(filename='saved_img.jpg', img=frame)
    q = cv2.waitKey(1)
    if q == ord("q"):
        break
cv2.destroyAllWindows()

In [7]:
# Reconocimiento del texto de la imagen con pytesseract.

im = Image.open("saved_img.jpg")

text = pytesseract.image_to_string(im, lang = 'spa')

print(text)

Tortilla York

York ham and potato
INGREDIENTES: PAN DÉ MOLDE 40% (harina de
trigo. agua, levadura, aceite E
sa, oluten de trigo, conservadores (E e
£-202), almidón de maíz, harina de
tratamiento de la harina (E-300)), Ti ROCA 3
(patata (50%), huevo líquido pasteurizado, ace
girasol, cebolla (7%) (cebolla, aceite de olivafsa
corrector acidez E-330) y Sal), FAMBRE PALET, Hi
(paleta de cerdo (45%) agua, almidón, aromas,
establlizantes (E-407. £-45, E-508, E-412), sal,
dextrosa, potenciador de y sabor (E-621), antioxidante
(E-316), colorante (E-20), conservador (E-250)),
- QUESO 6% (queso, grasas vegetales, agua, proteas
de la leche, almidón, gelztina, emulgentes (Exk7,
ES) corrector de acidez ( E-330), antioxidantes
(E-306, E 304) sales de fundido (E-452, E-339) sal,
conservador (£-202), colorante (E-160a) y aroma)
MAHONESA 4% (aceite de girasol, huevo entero,
vinagre de vino, agua, acidulante (acido cÍtrico),
azúcar, sal, establlizantes (E-41) F-£1 ) conservante
(E-202)). Puede contener

In [8]:
# Extracción de la info de los aditivos en el texto.

result = re.findall(r'E-\d+', text)


result2 = re.findall(r'E\d+', text)

valores = []

for value in result:
    valores.append(value)
    
for values in result2:
    valores.append(values)

print(valores)

['E-300', 'E-330', 'E-407', 'E-508', 'E-412', 'E-621', 'E-316', 'E-20', 'E-250', 'E-330', 'E-306', 'E-452', 'E-339', 'E-160', 'E-41', 'E-202']


In [9]:
# Homogeneización de los valores extraidos.

for value in valores:
    if "E-" in value:
        valores = [item.replace('E-','E') for item in valores]
    else:
        valores
        
valores

['E300',
 'E330',
 'E407',
 'E508',
 'E412',
 'E621',
 'E316',
 'E20',
 'E250',
 'E330',
 'E306',
 'E452',
 'E339',
 'E160',
 'E41',
 'E202']

In [10]:
# Busqueda de los aditivos que hemos extraido del texto para sacar la información.

alergias = aditivos[aditivos['id'].isin(valores)]
alergias

,id,tipo,origen,aditivo,classif
25,E160,No nocivo,Químico o sintético,Carotenoides,Colorante Colorantes
38,E202,Sospechoso,Químico o sintético,"Sorbato de potasio, Sorbato potásico, Sorbato ...",Conservante Conservantes
74,E250,¡Peligroso!,Químico o sintético,"Nitrito de sodio, Nitrito sódico, Nitrito de Na",Conservante Conservantes
91,E300,No nocivo,Químico o sintético,Ácido ascórbico,Antioxidante Antioxidantes
96,E306,No nocivo,Vegetal,Extracción Vegetal,Antioxidante Antioxidantes
104,E316,Sospechoso,Químico o sintético,"Eritorbato de sodio, Eritorbato sódico, Eritor...",Antioxidante Antioxidantes
111,E330,Sospechoso,Químico o sintético,Ácido cítrico,Antioxidante Antioxidantes
120,E339,Sospechoso,Químico o sintético,Sales ortofosfóricas,Antioxidante Antioxidantes
132,E407,¡Peligroso!,Indeterminado,"Carragenato, Carragenano, Carragenina",Estabilizante Estabilizantes
136,E412,No nocivo,Indeterminado,Goma guar,Estabilizante Estabilizantes


In [11]:
#  Reconocimiento de codigo de barras para su lectura.


url = 'http://192.168.43.203:8080/video'
cap = cv2.VideoCapture(url)
lastCode = ""

while True:

    _, frame = cap.read()
    
    
    frame = cv2.flip(frame, 1) 
    

    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB) 

    barcodes = decode(gray)    

    
    for barcode in barcodes:
       
        (x,y,w,h) = barcode.rect
        
        cv2.rectangle(frame, (x,y), (x+w,y+h), (0,0,255), 2)
        
        barcodeType = barcode.type
        barcodeData = barcode.data.decode("utf-8")
        
        dataText = f"Data: {barcodeData}"
        dataType = f"Type: {barcodeType}"
        cv2.putText(frame, dataText, (x,y-10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,0,255), 1)
        cv2.putText(frame, dataType, (x,y-30), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,0,255), 1)

        
        if barcodeData != lastCode:
            lastCode = barcodeData
            print("READ: " + barcodeData)

    cv2.imshow("original", frame)


    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()

READ: 8437000199281
READ: 8431000799281
READ: 8437000199281
READ: 8431000799281
READ: 8437000199281
READ: 8437000799887
READ: 8437000199281
READ: 8437000799887
READ: 8437000199281


In [12]:
# Extracción de datos sobre el producto a través de la API de "OpenFoodFacts" que funciona con códigos de barras.

URL = "https://world.openfoodfacts.org/api/v0/product/" + barcodeData +".json"

response = requests.get(URL)

contenido = response.content

res = json.loads(contenido)

print(res.get('product').get('nutrient_levels'))
nutri_score = res.get('product').get('nutrition_grades')

nutri_score = nutri_score.upper()

print("Nutri-Score:" + " " + nutri_score)

{'fat': 'moderate', 'salt': 'high', 'saturated-fat': 'moderate', 'sugars': 'low'}
Nutri-Score: D
